In [84]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import re

In [47]:
train_df = pd.read_csv('Train_data.csv',sep=",").copy()
train_txt = train_df.text.values
y_train = train_df.airline_sentiment.values
test_df = pd.read_csv('Test_data.csv',sep=",").copy()
test_txt = test_df.text.values
train_df

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [104]:
test_df.dtypes

tweet_id                   int64
airline                   object
airline_sentiment_gold    object
name                      object
negativereason_gold       object
retweet_count              int64
text                      object
tweet_coord               object
tweet_created             object
tweet_location            object
user_timezone             object
United                     int64
US Airways                 int64
American                   int64
Southwest                  int64
Delta                      int64
Virgin America             int64
c_dnt                     object
dtype: object

In [57]:
ind  = train_df.airline.value_counts().index
for i in ind:
    train_df[i]  =train_df['airline'].apply(lambda x: int(x==i))
    test_df[i] = test_df['airline'].apply(lambda x: int(x==i))

In [97]:
def dnt(s):
    pattern = re.compile(r'(-|:)')
    c_data = pattern.sub(' ',s)
    return c_data[:-5].strip()

In [98]:
train_df['c_dnt'] = train_df.tweet_created.apply(dnt)
test_df['c_dnt'] = test_df.tweet_created.apply(dnt)

In [105]:
train_dnt = train_df['c_dnt'].str.split(' ',expand=True).values
test_dnt = test_df['c_dnt'].str.split(' ',expand=True).values

In [106]:
train_dnt = train_dnt.astype('int64')
test_dnt = test_dnt.astype('int64')

In [99]:
test_df.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,United,US Airways,American,Southwest,Delta,Virgin America,c_dnt
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada),0,0,1,0,0,0,2015 02 22 18 15 50
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada),0,0,1,0,0,0,2015 02 22 13 22 57
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada),0,0,0,1,0,0,2015 02 17 18 52 31
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada),0,1,0,0,0,0,2015 02 22 23 16 24
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada),0,0,1,0,0,0,2015 02 23 08 44 51


In [5]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [6]:
def real_cleanser(splitted_tweet):
    Lemmatizer = WordNetLemmatizer()
    stops = set(stopwords.words('english'))
    stops.update(list(string.punctuation))
    cleaned_tweet = []
    for w in splitted_tweet:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = Lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            cleaned_tweet.append(clean_word.lower())
    return cleaned_tweet

In [7]:
def cleanser(tweets):
    splitted_tweets = []
    for i in tweets:
        splitted_tweets.append(word_tokenize(i))
    cleansed_tweets = [0]*len(tweets)
    for i in range(len(tweets)):
        if i%500==0:
            print(i)
        cleansed_tweets[i] = " ".join(real_cleanser(splitted_tweets[i]))
    return cleansed_tweets

In [8]:
x_train = cleanser(train_txt)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500


In [9]:
x_test = cleanser(test_txt)

0
500
1000
1500
2000
2500
3000
3500


In [114]:
count_vec = CountVectorizer(max_features=4000,ngram_range =(1,2))
x_train_features = count_vec.fit_transform(x_train)
x_test_features = count_vec.transform(x_test)

In [133]:
x_train_added = x_train_features.todense()
x_test_added = x_test_features.todense()

In [116]:
for i in ind:
    v_test = test_df[i].values
    v_train = train_df[i].values
    v_test = v_test.reshape(3660,1)
    v_train = v_train.reshape(10980,1)
    x_test_added = np.append(x_test_added,v_test,axis=1)
    x_train_added = np.append(x_train_added,v_train,axis=1)

In [134]:
x_test_added = np.append(x_test_added,test_dnt[:,:2],axis=1)
x_train_added = np.append(x_train_added,train_dnt[:,:2],axis=1)

In [135]:
x_test_added.shape

(3660, 4002)

In [136]:
clf = SVC()
clf.fit(x_train_added,y_train)
y_pred = clf.predict(x_test_added)

In [127]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
clf = SVC()
grid = {'C':[100,50,10,1],
       'gamma':['auto','scale',0.1,1,0.01]}
abc = GridSearchCV(clf,grid,verbose=4,n_jobs=3,cv=KFold(n_splits=2))
abc.fit(x_train_features,y_train)
print(abc.best_score_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done  40 out of  40 | elapsed:  3.8min finished


0.7688524590163934


In [138]:
y_pred = clf.predict(x_test_added)

In [129]:
abc.best_estimator_

SVC(C=10, gamma=0.01)

In [141]:
y_pred = y_pred.reshape(3660,1)
np.savetxt('Sentiment_analysis.csv',y_pred,delimiter=',',fmt="%s")

In [140]:
y_pred.shape

(3660,)